# TP1 Création d'un Environnement pour le Jeu du Pendu avec Python

Vous allez créer un environnement simulant le jeu du Pendu en python, à l'aide de la bibliothèque Gymnasium. L'objectif est de permettre à un agent d'apprendre à jouer au jeu du pendu en explorant différentes actions (deviner des lettres) et en recevant des récompenses en fonction de ses choix.

**Contexte du jeu :**

Dans le jeu du Pendu, l'agent doit deviner un mot en proposant des lettres une à une. Chaque mauvaise tentative réduit le nombre d'essais restants. Le but est de deviner toutes les lettres du mot avant d'épuiser les tentatives.

Vous allez implémenter un environnement où l'agent peut interagir, recevoir des observations, faire des actions (deviner une lettre) et recevoir des récompenses pour guider son apprentissage.

## 1. Création du fichier My_custom_Envs.py & import bibliothèques nécessaires

Nous allons utiliser les bibliothèques 
- Gymnasium pour créer un environnement personnalisable,  
- NumPy pour gérer les opérations mathématiques.
- nltk pour obtenir un dictionnaire de mots.

Créer un fichier My_custom_Envs.py pour construire l'environnement. 


## 2. Création de la class HangedManEnv de gym.Env

Nous définir ici la classe HangedManEnv, représentant un environnement de jeu du Pendu adapté pour des agents d'apprentissage par renforcement. 
Cet environnement doit être compatible avec la bibliothèque gymnasium pour faciliter l'entraînement d'agents intelligents à résoudre le jeu du Pendu.


Elle hérite de la classe gym.Env, ce qui en fait un environnement compatible avec Gym.
Dans cette classe, nous allons définir plusieurs méthodes dont 3 principales :
- __init__(), 
- __reset__(), 
- __step__()



In [1]:
import gymnasium as gym
class HangedManEnv_(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "n_total_try_games": 11}

    def __init__(self, max_word_size=8, word_dictionary=None, render_mode=None):
        pass

    def reset(self, seed=None, **kwargs):
        pass

    def step(self, action):
        pass

    def render(self):
        pass

### 2.1 Initialisation de la classe HangedManEnv : constructeur __init__

__Paramètres__ :
 - max_word_size=8 : la longueur maximale du mot à deviner.
 - word_dictionary=None : un dictionnaire de mots personnalisés. Si aucun n'est fourni, le dictionnaire par défaut sera utilisé.
 - render_mode=None : le mode de rendu pour l'affichage du jeu (par exemple, console, interface graphique).

__Attributs spécifiques :__

- n_total_try_games : le nombre total d'essais autorisés pour deviner le mot.
- encoded_state : un tableau NumPy vide qui représentera l'état encodé du mot à deviner.
- encoded_aim : un tableau NumPy vide qui représentera l'encodage du mot cible.
- encoded_tried_letters : une variable pour stocker les lettres déjà essayées.
- _nb_left_try : le nombre d'essais restants pour le joueur.
- word_dictionary : la liste des mots disponibles pour le jeu.
- max_word_size : la longueur maximale des mots à deviner.


__Définir un éspace d'action :__

L'agent peut choisir parmi les 26 lettres de l'alphabet anglais (de 'a' à 'z').
Utiliser spaces.Discrete(26) de la bibliothèque OpenAI Gym pour définir l'espace d'action.


__Définir un éspace d'observation :__

L'espace d'observation est un tuple composé  :

- du mot deviné encodé : Un espace spaces.MultiDiscrete([28] * max_word_size) :
Chaque position du mot peut prendre 28 états :
    - de 2 à 28 pour chaque lettre de l'alphabet.
    - 1 pour représenter une lettre inconnue (par exemple, '_').
    - 0 pour représenter une position non active dans le mot (utilisée comme remplissage si le mot est plus court que max_word_size).

-  des lettres essayées : Un espace spaces.MultiBinary(26) : Un vecteur binaire de taille 26 indiquant si chaque lettre de l'alphabet a déjà été essayée ou non.


__Chargement du dictionnaire de mots :__

Si aucun word_dictionary n'est fourni, utiliser le corpus de mots de la bibliothèque NLTK.

Assurez-vous que le corpus est téléchargé ; sinon, téléchargez-le en utilisant nltk.download('words').
Filtrer les mots pour qu'ils ne dépassent pas la longueur maximale spécifiée par max_word_size.




###  2.2 Méthodes d'encodage & Décodage :

Créer trois méthodes  pour gérer l'encodage et le décodage des mots et lettres essayées.

- encode_word(self, word) :

  - Prend en paramètre un mot (chaîne de caractères).
  - Retourne un tableau NumPy encoded de taille self.max_word_size et de type np.int8.
  - Pour chaque caractère du mot :
       - Si le caractère est '_', affecter la valeur 1 à la position correspondante dans encoded.
       - Sinon, encoder le caractère en utilisant la formule ord(char) - ord('A') + 2.
    _Les positions non utilisées du tableau sont remplies avec des zéros (0)._

- decode_word(encoded_word) (méthode statique) :

  - Prend en paramètre un tableau NumPy encodé.
  - Retourne le mot décodé sous forme de chaîne de caractères.
  - Pour chaque code dans encoded_word :
    - Si le code est 0, arrêter le décodage.
    - Si le code est 1, ajouter '_' au mot décodé.
    - Sinon, convertir le code en caractère avec chr(code + ord('A') - 2) et l'ajouter au mot décodé.


- decode_tried_letters(self) :

   - Parcourt self.encoded_tried_letters, une liste de booléens de taille 26 représentant les lettres de 'A' à 'Z'.
   - Pour chaque index où la valeur est True, convertir l'index en la lettre correspondante (chr(index + ord('A'))) et l'ajouter à un ensemble.
   - Retourne l'ensemble des lettres déjà essayées par le joueur.

### 2.3  La Class reset : Pour ré/initialiser l'environnement avant chaque épisode.

L'objectif de cette classe est : 
- de gérer les observations, les informations supplémentaires, 
- de réinitialiser les attributs de l'environnement pour démarrer de nouvelles parties. ( Nombre d'essai,  ...)
- de selectionner un nouveaux mots aléatoire depuis un dictionnaire donné, puis encoder ce mot dans un état cible.
- de préparer les espaces d'observation (encoded state, encoded aim, etc.) pour suivre l'état du jeu.
- Renvoie la prémière observation de la partie et des infos supplémentaires. 


__Méthode _get_obs(self) :__

_Retourne l'observation actuelle de l'environnement sous la forme d'un tuple contenant :_
  - self.encoded_state : le tableau NumPy représentant l'état encodé du mot à deviner.
  - self.encoded_tried_letters : le tableau NumPy des lettres déjà essayées.


__Méthode _get_info(self) :__

Retourne un dictionnaire avec des informations supplémentaires, notamment :
  - 'left try' : le nombre d'essais restants (self._nb_left_try).


__Méthode reset(self, seed=None, **kwargs) :__

_Réinitialise l'environnement pour commencer une nouvelle partie._

__Etapes de la méthode :__ 

- Initialiser self.encoded_state avec un tableau NumPy de zéros de taille self.max_word_size et de type np.uint8.
- Sélectionner aléatoirement un mot word dans self.word_dictionary, en s'assurant que sa longueur ne dépasse pas self.max_word_size. 
- Convertir le mot en majuscules (decoded_word).
- Activer les positions correspondantes aux lettres du mot dans self.encoded_state.
- Encoder le mot cible avec self.encode_word(decoded_word) et stocker le résultat dans self.encoded_aim.
- Initialiser self.encoded_tried_letters comme un tableau NumPy de booléens de taille 26, initialisé à False.
- Définir self._nb_left_try avec le nombre total d'essais autorisés (self.n_total_try_games).
- Si self.render_mode est "human", appeler self.render().
- __Retourner__ un tuple avec l'observation (self._get_obs()) et les informations (self._get_info()).

## 2.4 La méthode step(self, action) 

Cette méthode gère la logique d'un tour de jeu en fonction de l'action choisie par l'agent (une lettre de l'alphabet), met à jour l'état du jeu, calcule la récompense, et détermine si la partie est terminée.

__Paramètre :__

action : Un entier entre 0 et 25 inclus, représentant la lettre choisie par l'agent (0 pour 'A', 1 pour 'B', ..., 25 pour 'Z').

__Etapes de la méthode :__ 


- 1. Initialiser la récompense :
    Définir reward = -1 par défaut. Cela encourage l'agent à trouver le mot le plus rapidement possible, car chaque action coûte un point.
- 2. Si la lettre n'a pas encore été tentée 
    Marquer la lettre comme essayée :

## 2.5 render() : Pour afficher l'état actuel du jeu (le mot deviné et les lettres essayées).

## 3 Jouer avec l'environement.

Dans ce notebook nous allons seulement jouer avec l'envirronement que nous avons créé

1- Commencez par importer les bibliothéques necessaires random, gymnasium et numpy. 
 Importer aussi l'environnement HangedManEnv que l'on viens de créer. 

2 - Créer une instance de l'envirronement 

3 - Initialser l'envirronnement avec la méthode reset. 
   - Afficher le mot à deviner
   - Afficher le mot déviner décodé. 
   - Les lettres jouées codées 
   - les lettres restante à jouer et décodées
    
4 - Jouer une première action avec la methode step
    - Afficher le nombre d'essai restant et le mot deviné
    - Afficher les lettres restantes à jouer décodées

5 - Vérification 
   - Tester que le jeu s'arrette quand le bon mot est trouvé. 
   - Tester que si il y a une erreur le reward est actualiser et que si on se trompe plus autorisé le jeu s'arrette 





## 3 CORRECTION

### 1 - Importer les bibliothèques nécessaires

In [2]:
# imports 
import random
import gymnasium as gym
from gymnasium import spaces
import numpy as np

# On importe notre environnement 
from My_custom_Envs import HangedManEnv

### 2 - Créer une instance de l'environnement

In [3]:
env = HangedManEnv(max_word_size=8)

### 3 - Initialiser l'environnement avec la méthode reset

In [4]:
# Initialisation de l'environnement
observation, info = env.reset()

In [5]:
# Afficher le mot à deviner (encodé)
print( "Pour cette partie le mot à deviner est " + env.decode_word( env.encoded_aim ) )

Pour cette partie le mot à deviner est AETIAN


In [6]:
# Le mot déviné décodé , au premier tour toutes les lettres reste masquées
env.decode_word(observation[0])

'______'

In [7]:
# les lettres jouées codées: 
observation[1]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [8]:
## Les lettres jouées
[i for i , a in enumerate(observation[1])  if  a ]
env.decode_tried_letters()

set()

In [9]:
## Les lettres restantes à jouer.
possible_action = [i for i , a in enumerate(observation[1])  if not a ]
print(possible_action)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


In [10]:
## Les lettres restantes à jouer décodées. 
possible_action_decoded = [ chr( i +  ord('A')) for i , a in enumerate(observation[1])  if not a ]
print(possible_action_decoded)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


### 4 - Jouer une première action avec la methode step

In [11]:
### Essayons la lettre A 
obs, reward, terminated, truncated, info =  env.step(1)


In [12]:
env._nb_left_try, env.decode_word(observation[0])

(10, '______')

In [13]:
## Les lettres restantes à jouer 
possible_actions = [i for i , a in enumerate(observation[1])  if not a ]
print(possible_action)

env.close()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


### 5 - Vérification 
   - Tester que le jeu s'arrette quand le bon mot est trouvé. 


In [14]:
print(env.decode_word( env.encoded_aim ) )
set_actions = set(env.encoded_aim )
set_actions.discard(0)

set_actions

AETIAN


{2, 6, 10, 15, 21}

In [15]:
reward_cum = 0
terminated = False
step = 0
for action in set_actions :
    # Jouer l'action choisie
    obs, reward, terminated, _, info = env.step(action-2)
    step = step + 1
    reward_cum = reward_cum + reward
    if terminated : break
print(reward_cum, terminated)
print( step, len(set_actions))


0 True
5 5


 - Tester que si il y a une erreur le reward est actualisé

In [16]:
obs, info = env.reset()
print(env.decode_word( env.encoded_aim ) )

UNRHYMED


In [17]:
import string
set_alphabet = set(range(26))
set_actions = set(env.encoded_aim )
set_wrong_actions= set_alphabet-set_actions
print( set_wrong_actions)


{0, 1, 2, 3, 4, 7, 8, 10, 11, 12, 13, 16, 17, 18, 20, 21, 23, 24, 25}


In [18]:
reward_cum = 0
terminated = False
step = 0
for action in set_wrong_actions :
    # Jouer l'action choisie
    obs, reward, terminated, _, info = env.step(action-2)
    step = step + 1
    reward_cum = reward_cum + reward
    if terminated : break
print(reward_cum, terminated)
print( step, env.n_total_try_games)


letter out of bound : : 0 for A, 1 for B, ...
letter out of bound : : 0 for A, 1 for B, ...
-11 True
13 11


### Vérifier que les actions non possibles ne peuvent pas être jouées 

obs, info = env.reset()
print(env.decode_word( env.encoded_aim ) )
print( set(env.encoded_aim ) )

In [19]:
env.step(45)

letter out of bound : : 0 for A, 1 for B, ...


((array([1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8),
  array([ True,  True,  True, False, False,  True,  True, False,  True,
          True,  True,  True, False, False,  True,  True, False, False,
         False, False, False, False, False, False, False, False])),
 0,
 False,
 False,
 {'left try': 0})

In [20]:
a = env.encoded_aim[0]
a


22

In [21]:
env.step(-1)

letter out of bound : : 0 for A, 1 for B, ...


((array([1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8),
  array([ True,  True,  True, False, False,  True,  True, False,  True,
          True,  True,  True, False, False,  True,  True, False, False,
         False, False, False, False, False, False, False, False])),
 0,
 False,
 False,
 {'left try': 0})

In [22]:
env.step(46)

letter out of bound : : 0 for A, 1 for B, ...


((array([1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8),
  array([ True,  True,  True, False, False,  True,  True, False,  True,
          True,  True,  True, False, False,  True,  True, False, False,
         False, False, False, False, False, False, False, False])),
 0,
 False,
 False,
 {'left try': 0})